# Convolutional Neural Network

### Importing the libraries

In [1]:
!pip install --upgrade tensorflow keras Pillow

In [108]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
from tensorflow.keras.optimizers import SGD
from keras.layers import Dropout, Dense, Flatten



In [68]:
tf.__version__

'2.8.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [69]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (70,70),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [70]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (70,70),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [71]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [72]:
cnn.add(tf.keras.layers.Conv2D(filters=25, kernel_size=3,activation='relu', input_shape=[70,70,3]))

### Step 2 - Pooling

In [73]:
cnn.add(tf.keras.layers.MaxPooling2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [74]:
cnn.add(tf.keras.layers.Conv2D(filters=70, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPooling2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [80]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [82]:
cnn.add(tf.keras.layers.Dense(units=512, activation='tanh'))

### Step 5 - Output Layer

In [ ]:
cnn.add(Dropout(0.3))

In [91]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [94]:
cnn.compile(optimizer=SGD(momentum=0.5, decay=0.0004), loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [96]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25, batch_size=512)

Epoch 1/25
250/250 [==============================] - 51s 203ms/step - loss: 0.4090 - accuracy: 0.8116 - val_loss: 0.4708 - val_accuracy: 0.7850
Epoch 2/25
250/250 [==============================] - 52s 209ms/step - loss: 0.4070 - accuracy: 0.8115 - val_loss: 0.4542 - val_accuracy: 0.7920
Epoch 3/25
250/250 [==============================] - 53s 212ms/step - loss: 0.4040 - accuracy: 0.8106 - val_loss: 0.4358 - val_accuracy: 0.8015
Epoch 4/25
250/250 [==============================] - 53s 214ms/step - loss: 0.4065 - accuracy: 0.8101 - val_loss: 0.4371 - val_accuracy: 0.8010
Epoch 5/25
250/250 [==============================] - 54s 217ms/step - loss: 0.4046 - accuracy: 0.8110 - val_loss: 0.4343 - val_accuracy: 0.8055
Epoch 6/25
250/250 [==============================] - 54s 217ms/step - loss: 0.4087 - accuracy: 0.8094 - val_loss: 0.4581 - val_accuracy: 0.7875
Epoch 7/25
250/250 [==============================] - 53s 210ms/step - loss: 0.3979 - accuracy: 0.8188 - val_loss: 0.4416 - val_ac

## Part 4 - Making a single prediction

In [65]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (70, 70))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

In [66]:
print(prediction)

cat


## Fim